In [2]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# Import data from the clean file
df = pd.read_csv("data/Metadata-Clean.csv")
orig_df = pd.read_csv("data/movies_metadata.csv")
df["overview"], df["id"] = orig_df["overview"], orig_df["id"]
df

C:\Users\yis82\AppData\Local\Temp\ipykernel_6072\1957703653.py:9: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  orig_df = pd.read_csv("data/movies_metadata.csv")


,SampleID,Sample_Name,Proteomic_Data,Metabolomic_Data,Plate,Primer,Plate_Barcode,Total_ASVs,Category,Environment_vs_Host,...,WT_Time,Variation_By_Host,Animal_Tissues,Venom_Compartments,invivo_Treatment,Gland_Sites,Latitude,Longitude,overview,id
0,SUH125,MX_0_5_km_SU_16_Gland_1,Yes,No,Venomic_1,18S,GTACGAATCCT,27796.0,Venom,Venom,...,2016_Summer,NaN,NaN,NaN,NaN,PN_Minor,32.25,-116.95,"Led by Woody, Andy's toys live happily in his ...",862
1,SUH126,MX_0_5_km_SU_16_Gland_2,Yes,No,Venomic_1,18S,CTCAGTATGCA,1221.0,Venom,Venom,...,2016_Summer,NaN,NaN,NaN,NaN,PN_Minor,32.25,-116.95,When siblings Judy and Peter discover an encha...,8844
2,SUH127,MX_0_5_km_SU_16_Gland_3,Yes,No,Venomic_1,18S,GGTTGCACAAT,3844.0,Venom,Venom,...,2016_Summer,NaN,NaN,NaN,NaN,PN_Minor,32.25,-116.95,A family wedding reignites the ancient feud be...,15602
3,SUH129,MX_0_5_km_SU_16_Gland_5,Yes,No,Venomic_1,18S,CGATGTCGTCA,1582.0,Venom,Venom,...,2016_Summer,NaN,NaN,NaN,NaN,PN_Minor,32.25,-116.95,"Cheated on, mistreated and stepped on, the wom...",31357
4,SUH130,MX_0_5_km_SU_17_Gland_1,Yes,No,Venomic_1,18S,CAATTGTGCAC,2777.0,Venom,Venom,...,2017_Summer,NaN,NaN,NaN,NaN,PN_Minor,32.25,-116.95,Just when George Banks has recovered from his ...,11862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,SUH477,MX_0.5_km_SU_16_Sediment_2,NaN,NaN,Venomic_2,16S_V4,CGCGCAGTGTA,43780.0,Sediment,Sediment,...,2016_Summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Traces over three generations an immigrant fam...,38722
276,SUH478,MX_0.5_km_SU_16_Sediment_3,NaN,NaN,Venomic_2,16S_V4,CGCACATCAGA,43274.0,Sediment,Sediment,...,2016_Summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inspired by a true story. A petty criminal sen...,8438
277,SUH479,High_venom_-_triplicate_(#10)_b,NaN,NaN,Venomic_2,16S_V4,GTCACGACACA,3551.5,Control_Venom,Control_Venom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sully is a rascally ne'er-do-well approaching ...,11593
278,SUH481,High_water_-_triplicate_(#98)_b,NaN,NaN,Venomic_2,16S_V4,ACGATCGCTAG,27730.0,Control_Seawater1,Seawater,...,2017_Summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In a remote woodland cabin, a small town docto...",1945


In [4]:
from openai import OpenAI
import pandas as pd
import numpy as np
from dotenv import load_dotenv


load_dotenv()
# load_dotenv() is a function that loads variables from a .env file into environment variables in a Python script.
# We store OPENAI_API_KEY = xxx in .env file

client = OpenAI()

MODEL_NAME = "text-embedding-ada-002"


def get_embedding(text, model=MODEL_NAME):
    if not isinstance(text, str):
        text = str(text)

    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding


def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def search_docs(df, user_query, threshold=0.8):
    embedding = get_embedding(user_query, model=MODEL_NAME)
    df["similarities"] = df.embedding.apply(lambda x: cosine_similarity(x, embedding))

    # Filter results based on the threshold
    filtered_results = df[df["similarities"] > threshold]

    return filtered_results


df["embedding"] = df["overview"].apply(lambda x: get_embedding(x, model=MODEL_NAME))

title = "Toy Story"
description = df.loc[df["title"] == title, "overview"].iloc[0]
result = search_docs(df, description, threshold=0.8)

# remove the search item
print(result[result["title"] != title]["title"])

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable